In [2]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.1 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", openai_api_key="<>")

# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

Full result:
content='81 divided by 9 is 9.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 16, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BjcPUBcRqWqpROz80jcoJX0uF5TM4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a523e573-9054-4f02-a2a8-0ecf5f712608-0' usage_metadata={'input_tokens': 16, 'output_tokens': 9, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Content only:
81 divided by 9 is 9.


In [4]:
!pip install langchain_core

In [5]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
]

result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: 10 times 5 is 50.


In [6]:
!pip install langchain

In [7]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat_history = []

system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)

while True:
  query = input("You: ")
  if query.lower() == "exit":
    break
  chat_history.append(HumanMessage(content=query))

  result = model.invoke(chat_history)
  response = result.content
  chat_history.append(AIMessage(content=response))

  print(f"AI: {response}")

print("------Message History------")
print(chat_history)

You: god is dead as nietzsche said. is it true?
AI: The phrase "God is dead" originates from the German philosopher Friedrich Nietzsche and is found in several of his works, most notably in "The Gay Science" and "Thus Spoke Zarathustra." Nietzsche was not literally asserting that a deity had died; instead, he was using this phrase to express the idea that the traditional Christian God and, more broadly, the foundational religious and moral values of Europe, were losing their authority and influence in the modern world. This loss was due in large part to the rise of scientific understanding, secularization, and the Enlightenment's emphasis on reason and individualism.

The statement "God is dead" is meant as a metaphor for the decline of religious and metaphysical belief systems which had structured Western society and provided existential meaning. Nietzsche believed that without these structured beliefs, society would face a nihilistic crisis, where life is seen as lacking inherent mea

In [17]:
from langchain.prompts import ChatPromptTemplate

messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]

prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)


----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes.', additional_kwargs={}, response_metadata={})]


In [18]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt_template | model | StrOutputParser()

result = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(result)

Sure, here are three lawyer jokes for you:

1. Why don’t sharks attack lawyers?  
   Professional courtesy!

2. What's the difference between a lawyer and a herd of buffalo?  
   The lawyer charges more!

3. How many lawyer jokes are there, anyway?  
   Only three. The rest are true stories!


In [12]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence

format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)
response = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(response)

I can share some lawyer jokes with you. Let me give you a few laughs:

1. How many lawyer jokes are there, anyway? Only three. The rest are true stories.
   
2. What's the difference between a lawyer and a herd of buffalo? The lawyer charges more. 

3. Why don’t sharks attack lawyers? Professional courtesy.

Feel free to ask for more!


In [19]:
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

result = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(result)

Word count: 65
SURE! HERE ARE THREE LAWYER JOKES FOR YOU:

1. WHY DID THE LAWYER SHOW UP TO COURT WEARING A VEST MADE ENTIRELY OF WATCHES?
   - BECAUSE HE WANTED TO WIN HIS CASE WITH TIME WELL-SPENT!

2. WHAT'S THE DIFFERENCE BETWEEN A LAWYER AND A HERD OF BUFFALO?
   - THE LAWYER CHARGES MORE!

3. WHY DID THE SHARK REFUSE TO EAT THE LAWYER?
   - PROFESSIONAL COURTESY!


In [22]:
from langchain.schema.runnable import RunnableParallel

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert product reviewer."),
    ("human", "List the main features of the product {product_name}"),
])

def analyze_pros(features):
  pros_template = ChatPromptTemplate.from_messages(
      [
          ("system", "You are an expert product reviewer."),
          ("human", "Given these features: {features}, list the pros of these features."),
      ]
  )
  return pros_template.format_prompt(features=features)

def analyze_cons(features):
  cons_template = ChatPromptTemplate.from_messages(
      [
          ("system", "You are an expert product reviewer."),
          ("human", "Given these features: {features}, list the cons of these features."),
      ]
  )
  return cons_template.format_prompt(features=features)

def combine_pros_cons(pros, cons):
  return f"Pros:\n{pros}\n\nCons:\n{cons}"

pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros":pros_branch_chain, "cons":cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

result = chain.invoke({"product_name": "Ninetendo Switch 2"})
print(result)

Pros:
Based on the speculative features of the anticipated Nintendo Switch 2, here are some potential advantages if these features were to be implemented:

1. **Improved Performance**:
   - **Pros**: Enhanced graphics and processing power could lead to smoother gameplay experiences and more visually stunning games. This opens up possibilities for more complex and demanding titles, improving the overall gaming experience.

2. **Enhanced Display**:
   - **Pros**: A higher resolution, potentially OLED, screen can provide sharper image quality, richer colors, and better contrast, significantly enhancing the portable gaming experience. This could make it more appealing for handheld gaming.

3. **Longer Battery Life**:
   - **Pros**: Extended battery life ensures longer gaming sessions without needing to recharge, making the device more convenient and portable. This is especially beneficial for long trips or commutes.

4. **Backward Compatibility**:
   - **Pros**: Supporting existing Switch 

In [25]:
from langchain.schema.runnable import RunnableBranch

positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Generate a response addressing this postive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Generate a request for more details for this neutral feedback: {feedback}."),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Generate a message to escalate this feedback to a human agent: {feedback}."),
    ]
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Classify the sentiment of this feedback as positive, negative, neutral or escalate: {feedback}."),
    ]
)

branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser()
)

classification_chain = classification_template | model | StrOutputParser()

chain = classification_chain | branches

review = "The product is ok. It broke after just one use and the quality is very good."
result = chain.invoke({"feedback": review})
print(result)

Thank you for sharing your experience with us. We appreciate your feedback and are glad to hear that you found the quality of the product very good. However, we're concerned to learn that it broke after just one use. This isn't the experience we strive to provide, and we sincerely apologize for any inconvenience this may have caused.

Your feedback is important to us, and we take issues like this seriously. We will be escalating this matter for further investigation to ensure the durability of our products meets our standards. In the meantime, please reach out to our customer service team so we can assist you with a replacement or provide further support.

Thank you for helping us improve our products and services.
